### Shared Mobility Data Wrangling

1. **Data Loading**:
    - The notebook starts by defining functions to load data from CSV files for different services (`Cargoroo`, `OVFiets`, and `CROW`).
    - These functions handle reading files, adding metadata (e.g., service type, modality), and extracting timestamps from filenames.

2. **Combining Data**:
    - Data from multiple folders (representing different timestamps) is loaded and combined into a single dataframe using the `load_all_mobility_data` function.
    - This ensures all data collected over 24 hours is consolidated for analysis.

3. **Initial Data Inspection**:
    - The combined dataframe is inspected for missing values (`isnull().sum()`) and duplicate rows (`duplicated().sum()`).
    - Missing values in critical columns like `modality` are identified and reviewed.

4. **Data Cleaning**:
    - Duplicate rows are removed to ensure data integrity.
    - The `timestamp` column is converted to a consistent datetime format to facilitate time-based analysis.
    - Additional columns (`date_only` and `time_only`) are created by splitting the `timestamp` into separate date and time components.

5. **Data Export**:
    - The cleaned and enriched dataframe is saved to a CSV file for further use.
    - Separate CSV files for each service (`Cargoroo`, `OVFiets`, and `CROW`) can also be exported if needed.

6. **Geospatial Data Preparation**:
    - A GeoDataFrame is created by converting latitude and longitude into geometry points.
    - The data is reprojected to the Amersfoort / RD New coordinate system for geospatial analysis.
    - The geospatial data is exported as a GeoJSON file for use in GIS tools like QGIS.

This process ensures that the data collected over 24 hours is cleaned, structured, and ready for analysis or visualization.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import csv
import os
import glob
from datetime import date
from geopy.distance import geodesic


In [2]:

from config import RAW_DIR
folder_path = RAW_DIR / "CSV_vehicle_locations"
print(os.listdir(folder_path))



['09-04-2025', '10-04-2025', '11-04-2025']


### FUNCTIONS TO EXTRACT DATA FROM THE CSVs

In [ ]:
def load_cargoroo_data(folder_path):
    dfs = []
    files = glob.glob(os.path.join(folder_path, "Cargoroo_locations_*.csv"))
    print(f"📁 Cargoroo files matched: {len(files)}")

    for file in files:
        print(f"🔄 Reading: {os.path.basename(file)}")
        try:
            df = pd.read_csv(file)
            df["service"] = "Cargoroo"
            df["modality"] = df.get("modality", "cargobike")
            df["available"] = 1

            time_str = os.path.basename(file).replace("Cargoroo_locations_", "").replace(".csv", "")
            df["timestamp"] = pd.to_datetime(time_str, format="%d-%m-%Y_%H-%M")

            dfs.append(df[["service", "modality", "latitude", "longitude", "available", "timestamp"]])
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")

    if dfs:
        print(f"✅ Loaded {sum(len(df) for df in dfs)} rows for Cargoroo")
        return pd.concat(dfs, ignore_index=True)
    else:
        print("⚠️ No valid Cargoroo data found.")
        return pd.DataFrame()


In [4]:
#funtion OVfiets_data
# This function loads OVfiets data from CSV files in the specified folder path.
# It assumes that the CSV files have a specific naming convention and structure.
# The function reads each file, adds service and modality columns, and selects relevant columns.
# Finally, it concatenates all the dataframes into a single dataframe and returns it.  

def load_ovfiets_data(folder_path):
    dfs = []
    files = glob.glob(os.path.join(folder_path, "OVFiets_locations_*.csv"))
    print(f"📁 OVFiets files matched: {len(files)}")

    for file in files:
        print(f"🔄 Reading: {os.path.basename(file)}")
        try:
            df = pd.read_csv(file)
            df["service"] = "OVFiets"
            df["modality"] = "bicycle"

            time_str = os.path.basename(file).replace("OVFiets_locations_", "").replace(".csv", "")
            df["timestamp"] = pd.to_datetime(time_str, format="%d-%m-%Y_%H-%M")

            dfs.append(df[["service", "modality", "latitude", "longitude", "available", "timestamp"]])
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")

    if dfs:
        print(f"✅ Loaded {sum(len(df) for df in dfs)} rows for OVFiets")
        return pd.concat(dfs, ignore_index=True)
    else:
        print("⚠️ No valid OVFiets data found.")
        return pd.DataFrame()



In [5]:
def load_crow_data(folder_path):
    dfs = []
    files = glob.glob(os.path.join(folder_path, "CROW_locations_*.csv"))
    print(f"📁 CROW files matched: {len(files)}")

    for file in files:
        print(f"🔄 Reading: {os.path.basename(file)}")
        try:
            df = pd.read_csv(file)

            # Rename form_factor and system_id
            if "form_factor" in df.columns and "system_id" in df.columns:
                df = df.rename(columns={"form_factor": "modality", "system_id": "service"})
            else:
                print(f"⚠️ Missing expected columns in {file}, skipping.")
                continue

            # Clean up modality column (strip spaces + lowercase)
            df["modality"] = df["modality"].astype(str).str.strip().str.lower()

            # Normalize modality values
            modality_map = {
                "cargo_bicycle": "cargobike",
                "bicycle": "bicycle",
                "e-scooter": "scooter",
                "kick_scooter": "scooter",
                "e-bike": "bicycle"
            }

            df["modality"] = df["modality"].map(modality_map).fillna(df["modality"])

            df["available"] = 1

            # Extract timestamp from filename
            filename = os.path.basename(file)
            time_str = filename.replace("CROW_locations_", "").replace(".csv", "")
            df["timestamp"] = pd.to_datetime(time_str, format="%d-%m-%Y_%H-%M")

            dfs.append(df[["service", "modality", "latitude", "longitude", "available", "timestamp"]])

        except Exception as e:
            print(f"❌ Error reading {file}: {e}")

    if dfs:
        total_rows = sum(len(d) for d in dfs)
        print(f"✅ Loaded {total_rows} rows for CROW")
        return pd.concat(dfs, ignore_index=True)
    else:
        print("⚠️ No valid CROW data found.")
        return pd.DataFrame()


### Load the data per folder (date)

In [6]:
from config import RAW_DIR
folder_09 = RAW_DIR / "CSV_vehicle_locations" / "09-04-2025"
df_09_cargoroo = load_cargoroo_data(folder_09)
df_09_ovfiets = load_ovfiets_data(folder_09)
df_09_crow = load_crow_data(folder_09)




📁 Cargoroo files matched: 17
🔄 Reading: Cargoroo_locations_09-04-2025_10-14.csv
🔄 Reading: Cargoroo_locations_09-04-2025_10-35.csv
🔄 Reading: Cargoroo_locations_09-04-2025_10-37.csv
🔄 Reading: Cargoroo_locations_09-04-2025_10-56.csv
🔄 Reading: Cargoroo_locations_09-04-2025_11-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_12-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_13-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_14-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_15-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_16-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_17-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_18-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_19-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_20-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_21-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_22-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_23-00.csv
✅ Loaded 3026 rows for Cargoroo
📁 OVFiets files matched: 17
🔄 Reading: OVFiets_locations_09-04-2025_10-1

In [7]:
print(len(df_09_cargoroo), "rows in cargoroo")
print(len(df_09_ovfiets), "rows in ovfiets")
print(len(df_09_crow), "rows in crow")


3026 rows in cargoroo
850 rows in ovfiets
25052 rows in crow


In [8]:
from config import RAW_DIR
folder_10 = RAW_DIR / "CSV_vehicle_locations" / "10-04-2025"
df_10_cargoroo = load_cargoroo_data(folder_10)
df_10_ovfiets = load_ovfiets_data(folder_10)
df_10_crow = load_crow_data(folder_10)

📁 Cargoroo files matched: 9
🔄 Reading: Cargoroo_locations_10-04-2025_00-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_01-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_02-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_03-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_09-08.csv
🔄 Reading: Cargoroo_locations_10-04-2025_10-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_11-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_12-00.csv
🔄 Reading: Cargoroo_locations_10-04-2025_13-00.csv
✅ Loaded 1602 rows for Cargoroo
📁 OVFiets files matched: 9
🔄 Reading: OVFiets_locations_10-04-2025_00-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_01-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_02-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_03-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_09-08.csv
🔄 Reading: OVFiets_locations_10-04-2025_10-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_11-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_12-00.csv
🔄 Reading: OVFiets_locations_10-04-2025_13-00.csv
✅ Lo

In [9]:
print(len(df_10_cargoroo), "rows in cargoroo")
print(len(df_10_ovfiets), "rows in ovfiets")
print(len(df_10_crow), "rows in crow")

1602 rows in cargoroo
450 rows in ovfiets
14966 rows in crow


In [10]:
from config import RAW_DIR
folder_11 = RAW_DIR / "CSV_vehicle_locations" / "11-04-2025"
df_11_cargoroo = load_cargoroo_data(folder_11)
df_11_ovfiets = load_ovfiets_data(folder_11)
df_11_crow = load_crow_data(folder_11)

📁 Cargoroo files matched: 11
🔄 Reading: Cargoroo_locations_11-04-2025_06-30.csv
🔄 Reading: Cargoroo_locations_11-04-2025_07-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_08-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_09-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_10-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_11-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_12-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_13-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_14-00.csv
🔄 Reading: Cargoroo_locations_11-04-2025_16-23.csv
🔄 Reading: Cargoroo_locations_11-04-2025_17-00.csv
✅ Loaded 1958 rows for Cargoroo
📁 OVFiets files matched: 11
🔄 Reading: OVFiets_locations_11-04-2025_06-30.csv
🔄 Reading: OVFiets_locations_11-04-2025_07-00.csv
🔄 Reading: OVFiets_locations_11-04-2025_08-00.csv
🔄 Reading: OVFiets_locations_11-04-2025_09-00.csv
🔄 Reading: OVFiets_locations_11-04-2025_10-00.csv
🔄 Reading: OVFiets_locations_11-04-2025_11-00.csv
🔄 Reading: OVFiets_locations_11-04-2025_12-00.csv


In [11]:
print(len(df_11_cargoroo), "rows in cargoroo")
print(len(df_11_ovfiets), "rows in ovfiets")
print(len(df_11_crow), "rows in crow")

1958 rows in cargoroo
550 rows in ovfiets
17247 rows in crow


### Function to load all mobility data at once

In [12]:
# Load all mobility data from the specified root folder
# This function loads mobility data from multiple subfolders within a specified root folder.
# It processes each subfolder, loads data from Cargoroo, OVFiets, and CROW, and combines the data into a single dataframe.
# The function also handles errors during file reading and prints messages accordingly.
# The function is designed to be reusable, allowing it to be called multiple times with different root folders if needed.
# The function is also designed to be flexible, allowing it to handle different file formats and structures.
# The function is designed to work with multiple datasets, making it suitable for various data sources.
# The function is designed to be used in a larger data processing pipeline, where multiple datasets are loaded and combined for analysis.


def load_all_mobility_data(root_folder):
    all_data = []

    # Loop through each subfolder (e.g. "09-04-2025", "10-04-2025", ...)
    for folder_name in sorted(os.listdir(root_folder)):
        folder_path = os.path.join(root_folder, folder_name)

        if not os.path.isdir(folder_path):
            continue

        print(f"\n Processing folder: {folder_name}")

        df_cargoroo = load_cargoroo_data(folder_path)
        df_ovfiets = load_ovfiets_data(folder_path)
        df_crow = load_crow_data(folder_path)

        # Combine all dataframes for the day
        daily_df = pd.concat([df_cargoroo, df_ovfiets, df_crow], ignore_index=True)

        if not daily_df.empty:
            daily_df["date_folder"] = folder_name  # optional: adds the folder name as context
            all_data.append(daily_df)

    if all_data:
        full_df = pd.concat(all_data, ignore_index=True)
        print(f"\n Finished. Total rows combined: {len(full_df)}")
        return full_df
    else:
        print("\n No valid data found in any folder.")
        return pd.DataFrame()


In [13]:
from config import RAW_DIR
root_path = RAW_DIR / "CSV_vehicle_locations"
all_mobility_data = load_all_mobility_data(str(root_path))
print(all_mobility_data.head())
print(all_mobility_data.columns)


 Processing folder: 09-04-2025
📁 Cargoroo files matched: 17
🔄 Reading: Cargoroo_locations_09-04-2025_10-14.csv
🔄 Reading: Cargoroo_locations_09-04-2025_10-35.csv
🔄 Reading: Cargoroo_locations_09-04-2025_10-37.csv
🔄 Reading: Cargoroo_locations_09-04-2025_10-56.csv
🔄 Reading: Cargoroo_locations_09-04-2025_11-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_12-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_13-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_14-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_15-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_16-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_17-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_18-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_19-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_20-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_21-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_22-00.csv
🔄 Reading: Cargoroo_locations_09-04-2025_23-00.csv
✅ Loaded 3026 rows for Cargoroo
📁 OVFiets files matched: 17
🔄 Reading: O

### Clean the combined dataframe

In [14]:
all_mobility_data.isnull().sum()


service        0
modality       0
latitude       0
longitude      0
available      0
timestamp      0
date_folder    0
dtype: int64

In [15]:
missing_mobility = all_mobility_data['modality'].isna().sum() # Check for missing values in the 'modality' column
missing_mobility_rows = all_mobility_data[all_mobility_data['modality'].isna()]
print(missing_mobility_rows.head(10)) 

Empty DataFrame
Columns: [service, modality, latitude, longitude, available, timestamp, date_folder]
Index: []


In [16]:
all_mobility_data.duplicated().sum()


4076

In [17]:
dupes = all_mobility_data[all_mobility_data.duplicated()]
print(dupes.head())



      service modality   latitude  longitude  available           timestamp  \
3881  moveyou  bicycle  52.084479   5.179265          1 2025-04-09 10:35:00   
3882  moveyou  bicycle  52.084479   5.179265          1 2025-04-09 10:35:00   
3884  moveyou  bicycle  52.084224   5.167485          1 2025-04-09 10:35:00   
3885  moveyou  bicycle  52.084000   5.174200          1 2025-04-09 10:35:00   
3886  moveyou  bicycle  52.084000   5.174200          1 2025-04-09 10:35:00   

     date_folder  
3881  09-04-2025  
3882  09-04-2025  
3884  09-04-2025  
3885  09-04-2025  
3886  09-04-2025  


In [18]:
all_mobility_data = all_mobility_data.drop_duplicates()
print(dupes.head())

      service modality   latitude  longitude  available           timestamp  \
3881  moveyou  bicycle  52.084479   5.179265          1 2025-04-09 10:35:00   
3882  moveyou  bicycle  52.084479   5.179265          1 2025-04-09 10:35:00   
3884  moveyou  bicycle  52.084224   5.167485          1 2025-04-09 10:35:00   
3885  moveyou  bicycle  52.084000   5.174200          1 2025-04-09 10:35:00   
3886  moveyou  bicycle  52.084000   5.174200          1 2025-04-09 10:35:00   

     date_folder  
3881  09-04-2025  
3882  09-04-2025  
3884  09-04-2025  
3885  09-04-2025  
3886  09-04-2025  


In [19]:
all_mobility_data["service"].value_counts()


service
greenwheels           17757
mywheels              15919
check                 14002
Cargoroo               6586
OVFiets                1850
donkey                 1596
baqme                  1558
tier2                  1008
deelfietsnederland      540
felyx                   413
moveyou                 396
Name: count, dtype: int64

In [20]:
print(all_mobility_data["timestamp"].min())
print(all_mobility_data["timestamp"].max())


2025-04-09 10:14:00
2025-04-11 17:00:00


In [21]:
# Convert the timestamp column to datetime format
# This ensures that the timestamp is in a consistent format for further analysis.
all_mobility_data["timestamp"] = pd.to_datetime(all_mobility_data["timestamp"], errors="coerce")


In [22]:
all_mobility_data["timestamp"].isnull().sum()  # Check for any null values in the timestamp column after conversion

0

In [23]:
# Ensure timestamp is in datetime format
all_mobility_data["timestamp"] = pd.to_datetime(all_mobility_data["timestamp"], errors="coerce")

# Split into clean date and time columns
all_mobility_data["date_only"] = all_mobility_data["timestamp"].dt.date  # YYYY-MM-DD
all_mobility_data["time_only"] = all_mobility_data["timestamp"].dt.time  # HH:MM:SS

# Optional: drop the original messy date column
all_mobility_data.drop(columns=["date_folder"], inplace=True)


display(all_mobility_data[["timestamp", "date_only", "time_only"]].head())



,timestamp,date_only,time_only
0,2025-04-09 10:14:00,2025-04-09,10:14:00
1,2025-04-09 10:14:00,2025-04-09,10:14:00
2,2025-04-09 10:14:00,2025-04-09,10:14:00
3,2025-04-09 10:14:00,2025-04-09,10:14:00
4,2025-04-09 10:14:00,2025-04-09,10:14:00


### Save the cleaned data to a CSV files

In [ ]:
# Save the cleaned and enriched mobility data to CSV
from config import PROCESSED_DIR
output_path = PROCESSED_DIR / "combined_mobility_data_clean.csv"

# Export to CSV with proper datetime format
all_mobility_data.to_csv(output_path, index=False, date_format="%Y-%m-%d %H:%M:%S")

print("✅ Cleaned data exported to:", output_path)



#### <h4 style="color:red;">Change the boolean to True to run these cells:</h4>

In [ ]:
# Save individual file for Cargoroo
# change the boolean to True to export the file

if False:
    
    cargoroo_path = os.path.join(root_path, "cargoroo_data.csv")
    df_cargoroo = all_mobility_data[all_mobility_data["service"] == "Cargoroo"]
    df_cargoroo.to_csv(cargoroo_path, index=False)
    print("✅ Cargoroo data exported to:", cargoroo_path)



In [ ]:
# Save individual file for ovfiets

if False:

    ovfiets_path = os.path.join(root_path, "ovfiets_data.csv")
    df_ovfiets = all_mobility_data[all_mobility_data["service"] == "OVFiets"]       
    df_ovfiets.to_csv(ovfiets_path, index=False)
    print("✅ OVFiets data exported to:", ovfiets_path)

In [ ]:
# Save individual file for crow

if False:
    crow_path = os.path.join(root_path, "crow_data.csv")
    df_crow = all_mobility_data[all_mobility_data["service"] == "CROW"]
    df_crow.to_csv(crow_path, index=False)
    print("✅ CROW data exported to:", crow_path)

### Export GEODATA

In [ ]:
import geopandas as gpd

# Create geometry from lat/lon
all_mobility_data["geometry"] = gpd.points_from_xy(all_mobility_data["longitude"], all_mobility_data["latitude"])
all_mobility_gdf = gpd.GeoDataFrame(all_mobility_data, geometry="geometry")

# First set the original CRS to WGS84 (because it's lat/lon)
all_mobility_gdf.set_crs(epsg=4326, inplace=True)

# Then reproject to Amersfoort / RD New
all_mobility_gdf = all_mobility_gdf.to_crs(epsg=28992)

# Save to GeoJSON
from config import QGIS_DIR
output_geojson_path = QGIS_DIR / "shared_mobility_data_combined.geojson"
all_mobility_gdf.to_file(output_geojson_path, driver="GeoJSON")

print("✅ GeoJSON exported for QGIS:", output_geojson_path)
